In [7]:
import os
from datetime import datetime
import h5py
import numpy as np
import pandas as pd
from data_helpers import grouper, format_timeseries
from tempConv import tempConvDecoder
from sklearn.metrics import confusion_matrix


%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
conf = {
    'bs' : 256,
    'eps' : 25,
    'lr' : 0.0005,
    'kernel' : 2,
    'nb_filter' : 5,
    'window' : 60,
    'offset' : 10,
    'regressor' : True,
    'pyramidal' : True,
    'resample_data' : False,
    'sample_size' : 40000,
    'run_id' : datetime.now(),
    'verbose' : False,
    'key' : ['yaw_vel'],
    'dataset' : None
}

In [3]:
## get and format data
# lfp power bands
neural_data_file = h5py.File('datasets/GRat31_636061_all_sorted_spikes.hdf5', 'r')
neural_data = np.asarray(neural_data_file['sorted_spikes']) # iterate through powerbands
# print("neural_data.shape",neural_data.shape)

head_signals_h5 = h5py.File('datasets/GRat31_636061_all_head_data.hdf5', 'r')
idx_start, idx_stop = [3,6]

# dx
head_signal = np.asarray(
    [np.asarray(head_signals_h5[key]) for key in head_signals_h5.keys()][0:9]
).T[:,idx_start:idx_stop]
# print('head_signals shape: ', head_signal.shape)

# head_signals_keys = list(head_signals_h5.keys())[0:9][idx_start:idx_stop]
# head_signals_int = ['yaw_abs', 'roll_abs', 'pitch_abs']
# print('head_signals_keys intuitive: ', head_signals_int)

In [4]:
timeseries1 = neural_data
timeseries2 = head_signal[:,0]
if timeseries2.ndim == 1: 
    timeseries2 = np.atleast_2d(timeseries2).T
print('timeseries shapes:', timeseries1.shape, timeseries2.shape)

X_train, X_test, y_train, y_test = format_timeseries(
    timeseries1,
    timeseries2,
    conf['window'],
    conf['offset'],
    resample_data=conf['resample_data'],
    sample_size=conf['sample_size'],
    regressor=conf['regressor']
)

dataset = {
    'train' : (X_train, y_train),
    'test' : (X_test, y_test),
    'timeseries1' : timeseries1,
    'timeseries2' : timeseries2
}

conf['dataset'] = dataset

timeseries shapes: (80690, 34) (80690, 1)
X_train and y_train shape: (59536, 60, 34) (59536, 1)


In [5]:
stats = []

In [8]:
TCD = tempConvDecoder(**conf)
TCD.fit()
R2s,rs = TCD.determine_fit()
stats.append([R2s, rs])

Train on 59536 samples, validate on 14884 samples
Epoch 1/25
59536/59536 [==============================] - 1s - loss: 0.1042 - mean_squared_error: 0.0588 - val_loss: 0.1104 - val_mean_squared_error: 0.0714

In [9]:
print(stats)

[[[-0.00014879211544527315], [0.0]]]


In [44]:
TCD.model.evaluate(TCD.X_test,TCD.y_test)

24608/24913 [============================>.] - ETA: 0s

[0.52790511442393295, 0.74992975554931163]